# Apply Population Data as Edge Weights
This notebook assigns weights to the edges of the Donegal road graph using the coordinates of townlands found from querying the OSM API. The graph and townland population data must first be loaded from disk.

In [1]:
import config
import osmnx as ox
import networkx as nx
import pandas as pd

In [2]:
graph_file = f"{config.graph_graphml_path}/donegal_osm_simplified.graphml"
G = ox.load_graphml(graph_file)
population_data = pd.read_csv(f"{config.population_data_path}/donegal_townlands_all_coordinates.csv")
population_data.head()

,Unnamed: 0,townland,town,population,lat,lng
0,0,Abbey Island,Ballyshannon Rural,10.0,54.510056,-8.195737
1,1,Abbeylands,Ballyshannon Rural,121.0,54.513005,-8.204606
2,2,Adderville,Straid,54.0,55.239112,-7.434771
3,3,Adderwal,Gleann Léithín,9.0,54.905821,-8.168399
4,4,Admiran,Stranorlar,364.0,54.811157,-7.765547


Next the closest edges to each townland's coordinates are found

In [3]:
# get nearest edges to coordinates, returns an ndarray
nearest_edges = ox.get_nearest_edges(G, population_data["lng"], population_data["lat"], method="balltree")

In [4]:
# convert ndarray to list
nearest_edges_list = nearest_edges.tolist()
# get start (u) and end (v) points of each edge
start_points = [item[0] for item in nearest_edges_list]
end_points = [item[1] for item in nearest_edges_list]

# add these point to the population dataset
population_data["start"] = start_points
population_data["end"] = end_points

In [5]:
population_data.head()

,Unnamed: 0,townland,town,population,lat,lng,start,end
0,0,Abbey Island,Ballyshannon Rural,10.0,54.510056,-8.195737,1947246190,1947246217
1,1,Abbeylands,Ballyshannon Rural,121.0,54.513005,-8.204606,1947245998,1947245965
2,2,Adderville,Straid,54.0,55.239112,-7.434771,2120765476,2912801448
3,3,Adderwal,Gleann Léithín,9.0,54.905821,-8.168399,1902079898,1902080067
4,4,Admiran,Stranorlar,364.0,54.811157,-7.765547,1883609275,1883609441


In [6]:
# add default weight 1 to all edges
nx.set_edge_attributes(G, 1.0, "weight")
# check total weight of graph
G.size(weight='weight')

7252.0

In [7]:
# apply weights based on population values for closest edge to townland coordinates
for index, row in population_data.iterrows():
    weight_before = G[row["start"]][row["end"]][0]['weight']
    print(f"weight of edge {row['start']} - {row['end']} before: {weight_before}")
    G[row["start"]][row["end"]][0]["weight"] = float(weight_before + row["population"])
    print(f"weight of edge {row['start']} - {row['end']} after: "
          f"{G[row['start']][row['end']][0]['weight']}")

weight of edge 1947246190 - 1947246217 before: 1.0
weight of edge 1947246190 - 1947246217 after: 11.0
weight of edge 1947245998 - 1947245965 before: 1.0
weight of edge 1947245998 - 1947245965 after: 122.0
weight of edge 2120765476 - 2912801448 before: 1.0
weight of edge 2120765476 - 2912801448 after: 55.0
weight of edge 1902079898 - 1902080067 before: 1.0
weight of edge 1902079898 - 1902080067 after: 10.0
weight of edge 1883609275 - 1883609441 before: 1.0
weight of edge 1883609275 - 1883609441 after: 365.0
weight of edge 794807600 - 1885442121 before: 1.0
weight of edge 794807600 - 1885442121 after: 16.0
weight of edge 427999473 - 2417655996 before: 1.0
weight of edge 427999473 - 2417655996 after: 8.0
weight of edge 1329142156 - 1914032114 before: 1.0
weight of edge 1329142156 - 1914032114 after: 29.0
weight of edge 7475145002 - 427999446 before: 1.0
weight of edge 7475145002 - 427999446 after: 45.0
weight of edge 529112566 - 345006774 before: 1.0
weight of edge 529112566 - 345006774 a

weight of edge 1931036545 - 529650703 before: 1.0
weight of edge 1931036545 - 529650703 after: 45.0
weight of edge 1575258834 - 1574288969 before: 1.0
weight of edge 1575258834 - 1574288969 after: 52.0
weight of edge 320650871 - 320650854 before: 20.0
weight of edge 320650871 - 320650854 after: 34.0
weight of edge 1883123870 - 320650768 before: 1.0
weight of edge 1883123870 - 320650768 after: 301.0
weight of edge 1898178753 - 1886440530 before: 1.0
weight of edge 1898178753 - 1886440530 after: 25.0
weight of edge 777372756 - 777374369 before: 1.0
weight of edge 777372756 - 777374369 after: 32.0
weight of edge 777372756 - 777374369 before: 32.0
weight of edge 777372756 - 777374369 after: 608.0
weight of edge 777372756 - 777374369 before: 608.0
weight of edge 777372756 - 777374369 after: 697.0
weight of edge 1895997885 - 1895962395 before: 1.0
weight of edge 1895997885 - 1895962395 after: 39.0
weight of edge 1921559472 - 1918630786 before: 1.0
weight of edge 1921559472 - 1918630786 after

weight of edge 794807624 - 1885442530 before: 31.0
weight of edge 794807624 - 1885442530 after: 80.0
weight of edge 320650133 - 1880649412 before: 1.0
weight of edge 320650133 - 1880649412 after: 163.0
weight of edge 1230029853 - 33247197 before: 1.0
weight of edge 1230029853 - 33247197 after: 36.0
weight of edge 1921558719 - 790518165 before: 52.0
weight of edge 1921558719 - 790518165 after: 68.0
weight of edge 454697482 - 1879177039 before: 1.0
weight of edge 454697482 - 1879177039 after: 31.0
weight of edge 7475088185 - 427999473 before: 11.0
weight of edge 7475088185 - 427999473 after: 22.0
weight of edge 2010496124 - 401546727 before: 11.0
weight of edge 2010496124 - 401546727 after: 42.0
weight of edge 2054757241 - 911999962 before: 1.0
weight of edge 2054757241 - 911999962 after: 31.0
weight of edge 309927034 - 1984693487 before: 1.0
weight of edge 309927034 - 1984693487 after: 13.0
weight of edge 2295698412 - 52337817 before: 1.0
weight of edge 2295698412 - 52337817 after: 23.0

weight of edge 1415871578 - 898991000 before: 1.0
weight of edge 1415871578 - 898991000 after: 28.0
weight of edge 1979830363 - 2278757002 before: 1.0
weight of edge 1979830363 - 2278757002 after: 52.0
weight of edge 258431695 - 6988582874 before: 1.0
weight of edge 258431695 - 6988582874 after: 10.0
weight of edge 1947246114 - 52339425 before: 1.0
weight of edge 1947246114 - 52339425 after: 37.0
weight of edge 1514190567 - 529644993 before: 123.0
weight of edge 1514190567 - 529644993 after: 152.0
weight of edge 1911746300 - 1911746306 before: 34.0
weight of edge 1911746300 - 1911746306 after: 47.0
weight of edge 821222613 - 821243700 before: 1.0
weight of edge 821222613 - 821243700 after: 21.0
weight of edge 1880769527 - 262488338 before: 58.0
weight of edge 1880769527 - 262488338 after: 221.0
weight of edge 1875974694 - 1875974784 before: 1.0
weight of edge 1875974694 - 1875974784 after: 34.0
weight of edge 1930345675 - 529650641 before: 32.0
weight of edge 1930345675 - 529650641 aft

In [8]:
graph = list(G.edges(data=True))

G.size(weight="weight")

165172.0

In [9]:
ox.save_graphml(G, filepath=f"{config.graph_graphml_path}/donegal_osm_weights_applied.graphml")